# Aggregation Pipeline Validation

This notebook validates the data aggregation pipeline from raw plays to season-level team stats. It performs a series of checks to ensure that the aggregations are correct and that the resulting metrics are within expected ranges.

## 1. Setup and Configuration

In [4]:
import os
import sys
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Add src to path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import project modules
from cfb_model.data.storage.local_storage import LocalStorage
from cfb_model.data.aggregations.pipeline import build_preaggregation_pipeline

# Set display options for pandas
pd.set_option('display.max_columns', None)

## 2. Load Raw Plays Data

In [5]:
YEAR_TO_VALIDATE = 2023
DATA_ROOT = os.getenv("CFB_MODEL_DATA_ROOT")

raw_storage = LocalStorage(data_root=DATA_ROOT, file_format="csv", data_type="raw")
plays_raw_df = pd.DataFrame.from_records(raw_storage.read_index("plays", {"year": YEAR_TO_VALIDATE}))

print(f"Loaded {len(plays_raw_df):,} raw plays for {YEAR_TO_VALIDATE}.")
plays_raw_df.head()

Loaded 131,791 raw plays for 2023.


,season,week,id,game_id,play_number,period,clock_minutes,clock_seconds,wallclock,offense,offense_conference,offense_score,offense_timeouts,defense,defense_conference,defense_score,defense_timeouts,home,away,down,distance,yardline,yards_to_goal,yards_gained,play_type,play_text,ppa,scoring
0,2023,1,401520145102867313,401520145,10,2,13,26,NaN,UTEP,Conference USA,0,3.0,Jacksonville State,Conference USA,10,3.0,Jacksonville State,UTEP,1,10,NaN,31,0,Pass Incompletion,Gavin Hardison pass incomplete,-0.582484,False
1,2023,1,401520145102867314,401520145,11,2,13,26,NaN,UTEP,Conference USA,0,3.0,Jacksonville State,Conference USA,10,3.0,Jacksonville State,UTEP,2,10,NaN,31,4,Rush,Mike Franklin run for 4 yds to the JVST 27,-0.096200,False
2,2023,1,401520145102867315,401520145,12,2,13,26,NaN,UTEP,Conference USA,0,3.0,Jacksonville State,Conference USA,10,3.0,Jacksonville State,UTEP,3,6,NaN,27,1,Rush,Torrance Burgess Jr. run for 1 yd to the JVST 26,-0.647224,False
3,2023,1,401520145102937301,401520145,13,2,6,26,NaN,UTEP,Conference USA,0,3.0,Jacksonville State,Conference USA,10,3.0,Jacksonville State,UTEP,4,5,NaN,26,43,Field Goal Missed,Justin Duff 43 yd FG MISSED,NaN,False
4,2023,1,401520145102947503,401520145,1,2,5,24,NaN,UTEP,Conference USA,0,3.0,Jacksonville State,Conference USA,10,3.0,Jacksonville State,UTEP,1,10,NaN,71,11,Pass Reception,Gavin Hardison pass complete to Tyrin Smith fo...,1.018716,False


## 3. Run Aggregation Pipeline

In [6]:
byplay_df, drives_df, team_game_df, team_season_df = build_preaggregation_pipeline(plays_raw_df)
print("Aggregation pipeline complete.")

KeyError: "Column(s) ['is_busted_drive', 'is_explosive_drive', 'is_successful_drive'] do not exist"

## 4. Validation Checks

### 4.1. Play-to-Drive Aggregation

In [ ]:
# Check that the number of plays in the drives data matches the number of countable plays in the byplay data
byplay_play_counts = byplay_df[byplay_df['is_drive_play'] == 1].groupby(['game_id', 'drive_number']).size().reset_index(name='byplay_play_count')
drives_play_counts = drives_df[['game_id', 'drive_number', 'drive_plays']]

play_counts_validation = pd.merge(byplay_play_counts, drives_play_counts, on=['game_id', 'drive_number'])
play_counts_validation['play_count_diff'] = play_counts_validation['byplay_play_count'] - play_counts_validation['drive_plays']

print("Play Count Validation:")
print(play_counts_validation[play_counts_validation['play_count_diff'] != 0])

# Check that the total yards in the drives data matches the sum of yards_gained in the byplay data
byplay_yardage = byplay_df.groupby(['game_id', 'drive_number'])['yards_gained'].sum().reset_index(name='byplay_yards')
drives_yardage = drives_df[['game_id', 'drive_number', 'drive_yards']]

yardage_validation = pd.merge(byplay_yardage, drives_yardage, on=['game_id', 'drive_number'])
yardage_validation['yardage_diff'] = yardage_validation['byplay_yards'] - yardage_validation['drive_yards']

print("
Yardage Validation:")
print(yardage_validation[yardage_validation['yardage_diff'] != 0])

### 4.2. Drive-to-Game Aggregation

In [ ]:
# Check that the number of drives in the team_game data matches the number of drives in the drives data
drives_drive_counts = drives_df.groupby(['game_id', 'offense']).size().reset_index(name='drives_drive_count')
team_game_drive_counts = team_game_df[['game_id', 'team', 'off_drives']]

drive_counts_validation = pd.merge(drives_drive_counts, team_game_drive_counts, left_on=['game_id', 'offense'], right_on=['game_id', 'team'])
drive_counts_validation['drive_count_diff'] = drive_counts_validation['drives_drive_count'] - drive_counts_validation['off_drives']

print("Drive Count Validation:")
print(drive_counts_validation[drive_counts_validation['drive_count_diff'] != 0])

### 4.3. Game-to-Season Aggregation

In [ ]:
# Check that the games_played in the team_season data is correct
team_game_game_counts = team_game_df.groupby('team')['game_id'].nunique().reset_index(name='team_game_game_count')
team_season_game_counts = team_season_df[['team', 'games_played']]

game_counts_validation = pd.merge(team_game_game_counts, team_season_game_counts, on='team')
game_counts_validation['game_count_diff'] = game_counts_validation['team_game_game_count'] - game_counts_validation['games_played']

print("Game Count Validation:")
print(game_counts_validation[game_counts_validation['game_count_diff'] != 0])